# Quick method for removing events below a certain threshold of Nsipms

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
import os
import sys
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

from invisible_cities.io.dst_io        import df_writer
from invisible_cities.io.dst_io        import load_dst



In [ ]:
file_path = '/home/e78368jw/Downloads/test_data/sophronia_lowSiPM_test/'
minimum_SiPMs = 10


# Collect all .h5 files in the directory
h5_files = list(Path(file_path).glob('*.h5'))
# Print the list of .h5 files
print(h5_files)

[PosixPath('/home/e78368jw/Downloads/test_data/sophronia_lowSiPM_test/sophronia_kr_202602_102_ldc1_kr_202602_fast.h5'), PosixPath('/home/e78368jw/Downloads/test_data/sophronia_lowSiPM_test/sophronia_kr_202602_100_ldc1_kr_202602_fast.h5'), PosixPath('/home/e78368jw/Downloads/test_data/sophronia_lowSiPM_test/sophronia_kr_202602_101_ldc1_kr_202602_fast.h5'), PosixPath('/home/e78368jw/Downloads/test_data/sophronia_lowSiPM_test/sophronia_kr_202602_103_ldc1_kr_202602_fast.h5')]


In [ ]:
import h5py

# disgustingly hardcoded shite
with h5py.File(h5_files[1], "r") as f:
    def walk(name, obj):
        if len(name.split("/")) > 1:
            if not name.split("/")[1].startswith("_i_"):
                if isinstance(obj, (h5py.Group, h5py.Dataset)):
                    print(name)
        elif len(name.split("/")) == 1:
            next 
        else:
            if isinstance(obj, (h5py.Group, h5py.Dataset)):
                        print(name)

    f.visititems(walk)

DST/Events
Filters/s12_selector
Filters/valid_hit
MC/configuration
MC/event_mapping
MC/hits
MC/particles
MC/sns_positions
MC/sns_response
RECO/Events
Run/eventMap
Run/events
Run/runInfo
config/detsim
config/hypathia
config/sophronia


In [54]:
# ugly bespoke table extractor
import tables as tb


def generate_df_table(file_path):
    '''
    generate a dictionary of suitable shape for storing
    all the relevant components of each IC output file.

    This is useful for then using the writer to output the file
    with all the expected tables without having to write them manually
    '''

    nodes = {}

    def walk(node):
        # skip nodes starting with _i_
        if node._v_name.startswith("_i_"):
            return
        # only print groups with no children or leaves
        if isinstance(node, tb.Group):
            if not node._v_children:  # empty dict → no children
                nodes[str(node._v_pathname)] = str(node._v_pathname).split('/')
                #print(node._v_pathname)
        else:  # leaf (Array, Table, etc.)
            nodes[str(node._v_pathname)] = str(node._v_pathname).split('/')
            #print(node._v_pathname)

    with tb.open_file(file_path, "r") as f:
        for node in f.walk_nodes("/"):
            walk(node)

    
    # then read in each bit
    for keys in nodes.keys():
        #print(keys)
        loaded_data = load_dst(file_path, nodes[keys][1], nodes[keys][2])
        nodes[keys][0] = loaded_data


    return nodes




In [ ]:
# then read in each bit
for keys in nodes.keys():
    print(keys)
    loaded_data = load_dst(h5_files[1], nodes[keys][1], nodes[keys][2])
    nodes[keys][0] = loaded_data

print()


In [41]:
display(nodes['/DST/Events'][0])
print(nodes['/DST/Events'][1])

,event,time,s1_peak,s2_peak,nS1,nS2,S1w,S1h,S1e,S1t,...,Nsipm,DT,Z,Zrms,X,Y,R,Phi,Xrms,Yrms
0,101000,2.020016e+11,0,0,1,1,200.0,3.733392,6.772552,10100.0,...,20,505.381836,439.682159,4.476864,453.833240,-110.459862,467.082424,-0.238751,16.274117,15.882934
1,101001,2.020028e+11,0,0,1,1,300.0,4.412321,14.106652,10000.0,...,16,781.487732,679.894287,5.565539,-32.885055,217.176089,219.651725,1.721076,14.296383,17.346152
2,101002,2.020050e+11,0,0,1,1,350.0,5.156821,19.575819,10000.0,...,17,1078.497803,938.293030,6.392197,111.187974,53.749287,123.497981,0.450287,15.628426,17.598882
3,101003,2.020064e+11,0,0,1,1,225.0,6.391879,16.319544,10000.0,...,48577,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
4,101004,2.020092e+11,0,0,1,1,200.0,2.927832,9.994072,10025.0,...,15,303.468628,264.017700,3.380394,5.511469,-46.555905,46.881004,-1.452961,13.058337,13.617530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,101994,2.039896e+11,1,0,2,1,150.0,1.792730,5.117953,10275.0,...,19,1243.202759,1081.586426,7.042724,175.231387,-114.784014,209.478898,-0.579912,16.703250,18.746136
940,101995,2.039908e+11,0,0,1,1,225.0,3.122980,14.508593,10025.0,...,13,563.469360,490.218292,5.387110,86.141691,-257.910883,271.916191,-1.248448,14.674328,13.296852
941,101996,2.039935e+11,0,0,1,1,225.0,5.251654,22.719267,10050.0,...,16,688.428345,598.932617,5.329683,-273.034975,182.725042,328.536967,2.551813,16.297954,14.274192
942,101998,2.039963e+11,0,0,1,1,175.0,3.911545,14.721220,10025.0,...,14,916.482971,797.340149,6.421928,42.141584,-175.169967,180.167784,-1.334707,14.669028,16.481356


DST


In [66]:
def writer_wrapper(file_path, nodes):
    '''
    Takes a dictionary of dfs and writes them to a file using the file wrapper
    '''
    with tb.open_file(f'{file_path}', 'w', compression = tb.Filters(complib="zlib", complevel=4)) as h5out:
        for key, table in nodes.items():
            df_writer(h5out, table[0], table[1], table[2])

In [67]:

for files in h5_files:
    # generate the dictionary
    table = generate_df_table(files)
    # extract the dst 
    dst  = table['/DST/Events'][0]
    RECO = table['/RECO/Events'][0]
    print(f'Events in {str(files).split('/')[-1]} DST: {dst.event.nunique()}')
    print(f'Events in {str(files).split('/')[-1]} RECO: {RECO.event.nunique()}')

    #display(dst)
    #display(RECO)

    # find events with Nsipm < N
    low_sipm_evts = dst[dst.Nsipm < minimum_SiPMs].event.unique()
    # remove them from the RECO and DST tables
    dst = dst[~dst.event.isin(low_sipm_evts)]
    RECO = RECO[~RECO.event.isin(low_sipm_evts)]
    print(f'Events in {str(files).split('/')[-1]} DST: {dst.event.nunique()}')
    print(f'Events in {str(files).split('/')[-1]} RECO: {RECO.event.nunique()}')
    table['/DST/Events'][0]  = dst
    table['/RECO/Events'][0] = RECO 

    writer_wrapper(f'{str(files)[:-3]}_output.h5', table)

Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 DST: 937
Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 RECO: 937
Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 DST: 926
Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 RECO: 926


/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)


Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 DST: 932
Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 RECO: 932
Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 DST: 922
Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 RECO: 922


/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)


Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 DST: 923
Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 RECO: 922
Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 DST: 912
Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 RECO: 911


/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)


Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 DST: 923
Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 RECO: 923
Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 DST: 912
Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 RECO: 912


/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)


# dry run

In [3]:
import numpy as np
import pandas as pd

from pathlib import Path
import os
import sys
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

from invisible_cities.io.dst_io        import df_writer
from invisible_cities.io.dst_io        import load_dst

import tables as tb

def generate_df_table(file_path):
    '''
    generate a dictionary of suitable shape for storing
    all the relevant components of each IC output file.

    This is useful for then using the writer to output the file
    with all the expected tables without having to write them manually
    '''

    nodes = {}

    def walk(node):
        # skip nodes starting with _i_
        if node._v_name.startswith("_i_"):
            return
        # only print groups with no children or leaves
        if isinstance(node, tb.Group):
            if not node._v_children:  # empty dict → no children
                nodes[str(node._v_pathname)] = str(node._v_pathname).split('/')
                #print(node._v_pathname)
        else:  # leaf (Array, Table, etc.)
            nodes[str(node._v_pathname)] = str(node._v_pathname).split('/')
            #print(node._v_pathname)

    with tb.open_file(file_path, "r") as f:
        for node in f.walk_nodes("/"):
            walk(node)

    
    # then read in each bit
    for keys in nodes.keys():
        #print(keys)
        loaded_data = load_dst(file_path, nodes[keys][1], nodes[keys][2])
        nodes[keys][0] = loaded_data


    return nodes


def writer_wrapper(file_path, nodes):
    '''
    Takes a dictionary of dfs and writes them to a file using the file wrapper
    '''
    with tb.open_file(f'{file_path}', 'w', compression = tb.Filters(complib="zlib", complevel=4)) as h5out:
        for key, table in nodes.items():
            df_writer(h5out, table[0], table[1], table[2])


def remove_low_sipm_events(file_path, min_SiPMs):
    # Collect all .h5 files in the directory
    h5_files = list(Path(file_path).glob('*.h5'))


    for files in h5_files:
        # generate the dictionary
        table = generate_df_table(files)
        # extract the dst 
        dst  = table['/DST/Events'][0]
        RECO = table['/RECO/Events'][0]
        print(f'Events in {str(files).split('/')[-1]} DST: {dst.event.nunique()}')
        print(f'Events in {str(files).split('/')[-1]} RECO: {RECO.event.nunique()}')

        #display(dst)
        #display(RECO)

        # find events with Nsipm < N
        low_sipm_evts = dst[dst.Nsipm < minimum_SiPMs].event.unique()
        # remove them from the RECO and DST tables
        dst = dst[~dst.event.isin(low_sipm_evts)]
        RECO = RECO[~RECO.event.isin(low_sipm_evts)]
        print(f'Events in {str(files).split('/')[-1]} DST: {dst.event.nunique()}')
        print(f'Events in {str(files).split('/')[-1]} RECO: {RECO.event.nunique()}')
        table['/DST/Events'][0]  = dst
        table['/RECO/Events'][0] = RECO 

        writer_wrapper(f'{str(files)[:-3]}_output.h5', table)


In [4]:
file_path = '/home/e78368jw/Downloads/test_data/sophronia_lowSiPM_test/'
minimum_SiPMs = 10

remove_low_sipm_events(file_path, minimum_SiPMs)

Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 DST: 937
Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 RECO: 937
Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 DST: 926
Events in sophronia_kr_202602_102_ldc1_kr_202602_fast.h5 RECO: 926


/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)


Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 DST: 932
Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 RECO: 932
Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 DST: 922
Events in sophronia_kr_202602_100_ldc1_kr_202602_fast.h5 RECO: 922


/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)


Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 DST: 923
Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 RECO: 922
Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 DST: 912
Events in sophronia_kr_202602_101_ldc1_kr_202602_fast.h5 RECO: 911


/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)
/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/io/dst_io.py:73: UserWarning: dataframe contains strings longer than allowed
  warnings.warn('dataframe contains strings longer than allowed', UserWarning)


Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 DST: 923
Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 RECO: 923
Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 DST: 912
Events in sophronia_kr_202602_103_ldc1_kr_202602_fast.h5 RECO: 912
